# YOLO v4 demo (MS COCO trained)

Choose GPU for faster inference

**Runtime -> Change Runtime Type -> GPU**

In [1]:
#@title Print GPU info
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Wed Mar 24 09:17:28 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8    26W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
#@title Download darknet
# Clone darknet
!git clone https://github.com/AlexeyAB/darknet.git
# For downloading & trimming vids
!git clone https://github.com/ankandrew/YT-Downloader-Trimmer.git ytdownloader

!pip install -r /content/ytdownloader/requirements.txt

from ytdownloader.downloader import Downloader
from google.colab import files

%cd darknet

!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.weights

# Enable GPU & OPENCV
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

# Make
!make

!chmod 755 darknet

Cloning into 'darknet'...
remote: Enumerating objects: 1, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 14737 (delta 0), reused 0 (delta 0), pack-reused 14736
Receiving objects: 100% (14737/14737), 13.27 MiB | 7.90 MiB/s, done.
Resolving deltas: 100% (10025/10025), done.
Cloning into 'ytdownloader'...
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 29 (delta 13), reused 10 (delta 2), pack-reused 0
Unpacking objects: 100% (29/29), done.
     |████████████████████████████████| 1.9MB 5.8MB/s 
/content/darknet
--2021-03-24 09:17:56--  https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights
Resolving github.com (github.com)... 52.192.72.89
Connecting to github.com (github.com)|52.192.72.89|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github-releases.githubusercontent.com/75388965/ba4b6380-889c-11e

In [3]:
#@title Download YT video for making predictions
video_url = "https://www.youtube.com/watch?v=Zbm0Z6nw5sE" #@param {type:"string"}
video_quality = 720 #@param ["1080", "720", "480", "360", "240"] {type:"raw"}
#@markdown Only video w/o sound
only_video = True #@param {type:"boolean"}
#@markdown Trim video
do_trim = True #@param {type:"boolean"}
#@markdown Time in hh:mm:ss format
start = "00:13:57" #@param {type:"string"}
end = "00:15:00" #@param {type:"string"}

yt_d = Downloader(
    video_url, 'downloaded_vid.mp4', quality=video_quality, only_vid=only_video)
yt_d.download()

if do_trim:
  import os
  yt_d.trim(start, end, delete_original=False)
  os.rename('downloaded_vid_trimmed.mp4', 'downloaded_vid.mp4')

[youtube] Zbm0Z6nw5sE: Downloading webpage
[youtube] Downloading just video Zbm0Z6nw5sE because of --no-playlist
[download] Destination: downloaded_vid.mp4
[download] 100% of 922.33MiB in 01:42

[MoviePy] Running:
>>> /usr/bin/ffmpeg -y -i downloaded_vid.mp4 -ss 837.00 -t 63.00 -vcodec copy -acodec copy downloaded_vid_trimmed.mp4
... command successful.


In [ ]:
#@title Or upload video from PC
file_name = "Name of the file.mp4" #@param {type:"string"}

assert file_name, 'Fill in the name of the file to upload'

files.upload()

assert os.path.isfile(file_name), 'Uploaded file has different name'
os.rename(file_name, 'downloaded_vid.mp4')

### Yolo v4

In [4]:
#@title Run YOLO v4
!./darknet detector demo cfg/coco.data cfg/yolov4.cfg yolov4.weights downloaded_vid.mp4 -dont_show -out_filename res_yolov4.mp4

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
person: 58% 
person: 44% 
person: 30% 

FPS:7.5 	 AVG_FPS:7.9

 cvWriteFrame 
Objects:

traffic light: 91% 
traffic light: 89% 
traffic light: 89% 
traffic light: 76% 
traffic light: 73% 
traffic light: 69% 
traffic light: 61% 
traffic light: 57% 
traffic light: 56% 
traffic light: 54% 
traffic light: 53% 
traffic light: 31% 
traffic light: 31% 
truck: 34% 
truck: 26% 
bus: 56% 
car: 100% 
car: 100% 
car: 100% 
car: 97% 
car: 97% 
car: 96% 
car: 93% 
car: 38% 
car: 34% 
car: 33% 
car: 32% 
car: 28% 
person: 93% 
person: 93% 
person: 75% 
person: 66% 
person: 43% 

FPS:7.6 	 AVG_FPS:7.7

 cvWriteFrame 
Objects:

handbag: 28% 
traffic light: 92% 
traffic light: 90% 
traffic light: 90% 
traffic light: 76% 
traffic light: 73% 
traffic light: 72% 
traffic light: 71% 
traffic light: 63% 
traffic light: 62% 
traffic light: 60% 
traffic light: 47% 
traffic light: 38% 
traffic light: 32% 
truck: 25% 
bus: 75% 
car: 10

### Yolo v4 - tiny

In [6]:
#@title Run YOLO v4 tiny
!./darknet detector demo cfg/coco.data cfg/yolov4-tiny.cfg yolov4-tiny.weights downloaded_vid.mp4 -dont_show -out_filename res_yolov4_tiny.mp4

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
person: 46% 

FPS:31.2 	 AVG_FPS:33.2

 cvWriteFrame 
Objects:

traffic light: 87% 
traffic light: 43% 
car: 99% 
car: 98% 
car: 92% 
car: 52% 
car: 49% 
car: 44% 
car: 41% 
car: 34% 
car: 30% 
car: 28% 
car: 26% 
person: 34% 

FPS:31.1 	 AVG_FPS:33.2

 cvWriteFrame 
Objects:

traffic light: 91% 
traffic light: 55% 
traffic light: 29% 
car: 98% 
car: 98% 
car: 94% 
car: 70% 
car: 50% 
car: 42% 
car: 35% 
car: 35% 
car: 32% 
person: 28% 
person: 25% 

FPS:30.8 	 AVG_FPS:33.2

 cvWriteFrame 
Objects:

traffic light: 89% 
traffic light: 61% 
car: 98% 
car: 97% 
car: 95% 
car: 71% 
car: 55% 
car: 41% 
car: 37% 
car: 33% 
car: 28% 
person: 41% 

FPS:31.7 	 AVG_FPS:33.2

 cvWriteFrame 
Objects:

traffic light: 88% 
traffic light: 69% 
car: 98% 
car: 97% 
car: 92% 
car: 74% 
car: 62% 
car: 50% 
car: 47% 
car: 33% 
person: 34% 
person: 26% 

FPS:32.2 	 AVG_FPS:33.2

 cvWriteFrame 
Objects:

traffic light: 90% 
traffi

### Download processed vid

In [8]:
#@title Download video
option = "yolo v4 tiny" #@param ["yolo v4", "yolo v4 tiny"]
# Download file
if option == 'yolo v4':
  files.download('res_yolov4.mp4')
else:
  files.download('res_yolov4_tiny.mp4')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>